In [4]:
from kaggle.api.kaggle_api_extended import KaggleApi
import zipfile
import os

1. Get your kaggle.json file:
- Go to https://www.kaggle.com/account where account is ur account name
- Scroll to the API section
- Click Create New API Token
- This downloads kaggle.json to your computer (usually in Downloads folder)


2. Put kaggle.json in the right folder
- Move the file to this folder:
    C:\Users\Morad Elshorbagy\\.kaggle\

If the .kaggle folder doesn’t exist, create it manually

In [7]:
# No need to run this code block again!

api = KaggleApi()
api.authenticate()

# Download the ZIP file to current directory (no unzip)
api.dataset_download_files('rounakbanik/the-movies-dataset', path='.', unzip=False)

# Create a folder 'data' if it doesn't exist
output_folder = 'Data'
os.makedirs(output_folder, exist_ok=True)

# Extract only the files you want into 'data' folder
with zipfile.ZipFile('the-movies-dataset.zip', 'r') as zip_ref:
    zip_ref.extract('ratings.csv', path=output_folder)
    zip_ref.extract('movies_metadata.csv', path=output_folder)

print(f"Files extracted to '{output_folder}/'")


Dataset URL: https://www.kaggle.com/datasets/rounakbanik/the-movies-dataset
Files extracted to 'Daat/'
